# Testing Orbit Controller SDK
    

### Author: AWS Professional Services Emerging Technology and Intelligent Platforms Group
### Date: Mar 10 2021

In [ ]:
import os
import sys
import boto3
from aws_orbit_sdk import controller
from aws_orbit_sdk.common import get_workspace,get_scratch_database
from aws_orbit_sdk.magics.orbit import OrbitWorkbenchMagics 

import json
import time

env_name = %env AWS_ORBIT_ENV
team_name = %env AWS_ORBIT_TEAM_SPACE
(env_name,team_name)

In [ ]:
%%run_notebook
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "ttt",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}

In [ ]:
job_name=_['Identifier']
job_name

In [ ]:
%%schedule_notebook -cron 0/2 * 1/1 * ?  -id ttt
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "ttt",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}


In [ ]:
cronjob_name=_['Identifier']
cronjob_name

In [ ]:
jobs = controller.list_my_running_jobs()
assert(len(jobs) > 0)
jobs

In [ ]:
jobs = controller.list_team_running_jobs()
print(jobs)
# assert(len(jobs) == 0)

In [ ]:
cronjobs = controller.list_running_cronjobs()
assert(len(cronjobs) > 0)
cronjobs

In [ ]:
!kubectl get cronjob 

In [ ]:
controller.delete_cronjob(job_name=cronjob_name)

In [ ]:
!kubectl get cronjob 

In [ ]:
!kubectl describe job $job_name

In [ ]:
job_name

In [ ]:
pods = controller.list_current_pods(f'job-name={job_name}')
assert(len(pods)==1)
pod_id = pods[0]['metadata']['name']
print(pod_id)
pods

In [ ]:
!kubectl wait --for=condition=Ready --timeout=120s pod/$pod_id

In [ ]:
controller.delete_job(job_name)

In [ ]:
!kubectl wait --for=delete --timeout=120s pod/$pod_id

In [ ]:
pod_id,job_name

In [ ]:
pods = controller.list_current_pods(f'job-name={job_name}')
pods

In [ ]:
assert(len(pods)==0)

In [ ]:
%%run_notebook
{    
      "tasks":  [
            {
                  "notebookName": "Example-1-SQL-Analysis-Athena.ipynb",
                  "sourcePath": "shared/samples/notebooks/B-DataAnalyst",
                  "targetPath": "shared/regression/notebooks/B-DataAnalyst",
                  "targetPrefix": "ttt",
                  "params": {
                        "glue_db" : "cms_raw_db",
                        "target_db" : "users"
                  }      
            }
      ]  
}

In [ ]:
!kubectl get jobs

In [ ]:
controller.delete_all_my_jobs()

In [ ]:
output = !kubectl get jobs
print(output)
assert('No resources found in lake-user namespace.' == output[0])